# 08. 지역 선형 임베딩 (LLE)

비선형 차원 축소 방법인 LLE를 구현합니다.

## 학습 목표
- 매니폴드 학습의 개념 이해
- 지역 이웃 구조 보존
- PCA와의 비교

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)

## 1. 스위스롤 데이터 생성

3D 매니폴드 데이터로 비선형 차원축소를 테스트합니다.

In [ ]:
# 스위스롤 데이터 생성
n_samples = 1000

t = 1.5 * np.pi * (1 + 2 * torch.rand(n_samples))
height = 10 * torch.rand(n_samples)

X = torch.stack([
    t * torch.cos(t),
    height,
    t * torch.sin(t)
], dim=1)

# 색상은 t값 (언롤링 시 x축이 됨)
colors = t.numpy()

# 3D 시각화
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=colors, cmap='viridis', s=10)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Swiss Roll (3D Manifold)')
plt.show()

## 2. PCA의 한계

In [ ]:
from mlfs.classical.reduction import PCA

# PCA로 2D 축소
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(10, 8))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=colors, cmap='viridis', s=10)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA (Fails to Unroll)')
plt.colorbar(label='t value')
plt.show()

## 3. LLE 적용

LLE 알고리즘:
1. 각 점에 대해 K개의 이웃 찾기
2. 이웃들의 선형 조합으로 각 점을 재구성하는 가중치 계산
3. 저차원에서 동일한 가중치로 재구성 오차를 최소화

In [ ]:
from mlfs.classical.reduction import LLE

# LLE 적용
lle = LLE(n_components=2, n_neighbors=12)
X_lle = lle.fit_transform(X)

plt.figure(figsize=(10, 8))
plt.scatter(X_lle[:, 0], X_lle[:, 1], c=colors, cmap='viridis', s=10)
plt.xlabel('LLE 1')
plt.ylabel('LLE 2')
plt.title('LLE (Successfully Unrolls Swiss Roll)')
plt.colorbar(label='t value')
plt.show()

## 4. 이웃 수의 영향

In [ ]:
# 다양한 이웃 수로 실험
n_neighbors_list = [5, 10, 20, 50]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for ax, n_neighbors in zip(axes, n_neighbors_list):
    lle = LLE(n_components=2, n_neighbors=n_neighbors)
    X_reduced = lle.fit_transform(X)
    
    ax.scatter(X_reduced[:, 0], X_reduced[:, 1], c=colors, cmap='viridis', s=5)
    ax.set_title(f'n_neighbors = {n_neighbors}')
    ax.set_xlabel('LLE 1')
    ax.set_ylabel('LLE 2')

plt.suptitle('Effect of Number of Neighbors', fontsize=14)
plt.tight_layout()
plt.show()

## 5. S-Curve 데이터

In [ ]:
# S-Curve 데이터 생성
n = 800
t_s = 3 * np.pi * (torch.rand(n) - 0.5)
height_s = 2 * torch.rand(n)

X_s = torch.stack([
    torch.sin(t_s),
    height_s,
    torch.sign(t_s) * (torch.cos(t_s) - 1)
], dim=1)

colors_s = t_s.numpy()

# 3D 시각화
fig = plt.figure(figsize=(12, 5))

ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(X_s[:, 0], X_s[:, 1], X_s[:, 2], c=colors_s, cmap='viridis', s=10)
ax1.set_title('S-Curve (3D)')

# PCA
ax2 = fig.add_subplot(132)
pca_s = PCA(n_components=2)
X_pca_s = pca_s.fit_transform(X_s)
ax2.scatter(X_pca_s[:, 0], X_pca_s[:, 1], c=colors_s, cmap='viridis', s=10)
ax2.set_title('PCA')

# LLE
ax3 = fig.add_subplot(133)
lle_s = LLE(n_components=2, n_neighbors=12)
X_lle_s = lle_s.fit_transform(X_s)
ax3.scatter(X_lle_s[:, 0], X_lle_s[:, 1], c=colors_s, cmap='viridis', s=10)
ax3.set_title('LLE')

plt.tight_layout()
plt.show()

## 요약

1. **LLE**: 지역 이웃 구조를 보존하는 비선형 차원축소
2. **핵심 아이디어**: 각 점은 이웃들의 선형 조합으로 표현 가능
3. **장점**: 비선형 매니폴드 펼치기 가능
4. **단점**: 이웃 수 선택 민감, 계산 복잡도

다음: 측지선 거리를 이용한 **Isomap**